In [2]:
import numpy as np
from random import Random
from math import ceil


# Assignment 2

The living area of an apartment influences its selling price.
Assume this relationship can be modeled by the following equation

Y = aX<sup>5</sup> + bX<sup>4</sup> + cX<sup>3</sup> + dX<sup>2</sup> + eX + f

Where the selling price is Y and X is the are of an appartment,

Let the loss function be the mean square error.
Use SGD to find optimal values of the hyper paramters.

We use the model (function defines the problem) $y = f(x)$ the selling price here is y and x is the area. as the model has unkown parameters (a, b, c, d,e ,f) the task is to derive these paramters using the known data (y,x)

In [8]:
SEED = 5
def generate_pnts(N=100):
    np.random.seed = 5
    x, y = np.array([]), np.array([])
    x = np.random.rand(N,6) #the dimension of the data x is (n,6)
    y = np.random.rand(N,1)  #the dimension of the data x is (n,1)

    return x, y
data_x, data_y = generate_pnts()

print(data_x.shape, data_y.shape)

(100, 6) (100, 1)


Knowing that the the model is $y = ax^5 +bx^4 + cx^3 +dx^2+ex + f$ <br>
it's can be written in teh vector form : $Y = P . X^T $ P is the parameters vector (1,6) and X is the data matrix (N,6) <br>
the answer is the Y vector predicted values (1,N)

In [14]:
def predict(data_x, params):
    # data_X, params and yhat should be numpy arrays
    # Y = aX^5 + bX^4 + cX^3 + dX^2 + eX + f
    data_x = np.array(data_x)
    yhat = np.dot(params, data_x.transpose()) #(1,6) dot (6,N) = (1,N)

    return yhat

In [15]:
params = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]) #shape of [[X]] is (X,1) but shape of [X] is (X,)

yhat = predict(data_x, params)
print(yhat.shape)

(1, 100)


Using the Mean square error:
- Mean squared Error:  $ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $
<br>

In [16]:
def loss(data_x, params, data_y):
      '''
      data_x shape is (N,6)
      data_y shape is (N,1)
      yhat shape is (N,1) after transposing
      param shape should be (1,6)
      '''
      yhat = predict(data_x, params).transpose() #shape of yhat after transposing is (N,1)
      loss = (1 / len(data_y)) * np.sum((yhat - data_y) ** 2) # ( (N,1) - (N,1) )^2 -> (N,1) the summation then is one value
      return loss
print(loss(data_x, params, data_y))

6.8191269002534005


We will use the closed form method to calculate the  gradient as the gradient of the MSE  with respect to its parameters is : $$\nabla_{\theta} J(\theta) = \begin{bmatrix}
\frac{\partial J}{\partial \theta_1} \\
\frac{\partial J}{\partial \theta_2} \\
\vdots \\
\frac{\partial J}{\partial \theta_m}
\end{bmatrix}$$

Or simply can express it as $$\frac{\partial \, \text{MSE}}{\partial {a}} =   -x^5 .2(y_i - \hat{y}_i)$$

apply the same principle for the rest prarmeters:
Apply the vector form:
$$\frac{\partial \, \text{L}}{\partial {P}} =   (Y- \hat{Y}_i)^{T}_{(1,N)}.  X_{(N,6)} . $$


In [17]:
def calc_grad(params, data_x, data_y):
    # write code here that calculates the gradient with respect to
    # each parameter
    # return one numpy array with the gradients
    data_x = np.array(data_x)

    yhat = predict(data_x, params).transpose()#shape of yhat after transposing is (N,1)
    loss_grad = (-2 * (data_y - yhat)) / len(data_y) #(N,1) - (N,1) -> (N,1)
    grad = np.dot(loss_grad.transpose(),data_x) #(N,1)^T dot (N,6) -> (1,6)

    return grad


We will divide the data into patches and loop over them to train the model. The update will be in vector form as wee discussed
$$P_{(1,6) }- \frac{\partial \, \text{L}}{\partial {P}}_{(1,6)}= P_{new(1,6) }$$

---



In [18]:
def train(data_x, data_y, EPOCHS = 100):
    # initialization
    params = np.array([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]) #shape of params must be (1,6)
    DELTA = 0.01
    BATCH_SIZE = 4
    indices = [i for i in range(len(data_x))]
    sampler = Random(x = 5)

    NUM_BATCHES = ceil(len(data_x)/BATCH_SIZE)

    batches_losses = []
    ##############################################

    # Optimization loop of SGD
    for _ in range(EPOCHS):
      sampler.shuffle(indices)

      batches_count = ceil(len(data_y) / BATCH_SIZE)

      for batch_index in range(batches_count): #looping over batches
          batch_indices = indices[batch_index * BATCH_SIZE: (batch_index + 1) * BATCH_SIZE]
          batch_x = [data_x[i] for i in batch_indices]
          batch_y = [data_y[i] for i in batch_indices]

          grad = calc_grad(params, data_x=batch_x, data_y=batch_y)
          params -= DELTA * grad #update the params  in vector form (1,6) - (1,6) -> (1,6)

          batches_losses.append(loss(data_x, params, data_y))
      return params, batches_losses

    # Don't change the return


In [19]:
print(train(data_x, data_y))

(array([[0.53354342, 0.5474588 , 0.52700217, 0.53714744, 0.50113882,
        0.5605846 ]]), [6.280311280125159, 5.933807619628513, 5.554135059461035, 5.298763231678374, 5.163589771515892, 4.925352447324985, 4.777890803874532, 4.368684298535331, 4.015190991335352, 3.6900656454812686, 3.495497790345883, 3.278731960972186, 3.095262153674454, 2.910760678974012, 2.688826457344556, 2.4897371527076944, 2.3504197926520884, 2.217477057530699, 2.0895591159745335, 2.0098174251599756, 1.8589227642870796, 1.7426183186984225, 1.652796720279795, 1.5338886872706263, 1.4481193035337512])
